# 전처리 - 동아리이름, 카테고리 분리

In [1]:
import random
import ast
import pandas as pd
import numpy as np

data = pd.read_csv('tag.csv',encoding='cp949',header=None)
data = data.iloc[400:600]

In [2]:
tags = data.iloc[:,6].apply(lambda x: "[" + x + ']' if x[0]=="'" else "['" + x + ']' )
tags

400               ['수레바퀴', '학회', '경영']
401      ['Young Tigers', '운동', '동아리']
402             ['KULSOM', '학회', '경영']
403               ['유도부', '동아리', '운동']
404           ['KU-HOPE', '봉사', '동아리']
                    ...               
595                ['명중', '동아리', '운동']
596           ['예술비평연구회', '예술', '동아리']
597            ['CREAKET', '경영', '학회']
598    ['Partyproviders', '동아리', '예술']
599              ['KUHS', '봉사', '동아리']
Name: 6, Length: 200, dtype: object

In [3]:
tags = tags.apply(ast.literal_eval)
tags

400               [수레바퀴, 학회, 경영]
401      [Young Tigers, 운동, 동아리]
402             [KULSOM, 학회, 경영]
403               [유도부, 동아리, 운동]
404           [KU-HOPE, 봉사, 동아리]
                 ...            
595                [명중, 동아리, 운동]
596           [예술비평연구회, 예술, 동아리]
597            [CREAKET, 경영, 학회]
598    [Partyproviders, 동아리, 예술]
599              [KUHS, 봉사, 동아리]
Name: 6, Length: 200, dtype: object

In [4]:
tags = tags.reset_index().iloc[:,1]

In [5]:
tags

0                 [수레바퀴, 학회, 경영]
1        [Young Tigers, 운동, 동아리]
2               [KULSOM, 학회, 경영]
3                 [유도부, 동아리, 운동]
4             [KU-HOPE, 봉사, 동아리]
                 ...            
195                [명중, 동아리, 운동]
196           [예술비평연구회, 예술, 동아리]
197            [CREAKET, 경영, 학회]
198    [Partyproviders, 동아리, 예술]
199              [KUHS, 봉사, 동아리]
Name: 6, Length: 200, dtype: object

## 동아리이름

In [6]:
names = pd.Series([_[0] for _ in tags])
names

0                수레바퀴
1        Young Tigers
2              KULSOM
3                 유도부
4             KU-HOPE
            ...      
195                명중
196           예술비평연구회
197           CREAKET
198    Partyproviders
199              KUHS
Length: 200, dtype: object

In [7]:
names.name = 'clubName'

## 카테고리 태그들

In [8]:
categorytags = pd.Series([_[1:] for _ in tags])
categorytags

0       [학회, 경영]
1      [운동, 동아리]
2       [학회, 경영]
3      [동아리, 운동]
4      [봉사, 동아리]
         ...    
195    [동아리, 운동]
196    [예술, 동아리]
197     [경영, 학회]
198    [동아리, 예술]
199    [봉사, 동아리]
Length: 200, dtype: object

In [9]:
categorytags.name = 'categories'

## 조회수 랜덤 부여
태그를 기반으로 동아리를 뽑으면, 랭킹별로 나열할 예정
(조회수는 어플 제작하면 db에 실시간 로그 수집할 예정이나, 우선은 랜덤으로 조회수 부여)  
DB에는 유저별-페이지별 뷰로 수집(collaborative filtering용)이 될건데, 여기서는 페이지별로 다 sum aggregate 할것임

In [10]:
## 조회수 랜덤 부여
import random
lst = []
for _ in range(200):
    lst.append(random.randint(0,200))
viewed = pd.Series(lst)
viewed

0       74
1      108
2       98
3       86
4      119
      ... 
195    176
196    195
197      8
198    170
199      9
Length: 200, dtype: int64

In [11]:
viewed.name = 'viewed'

In [12]:
data = data.iloc[:,:-1].reset_index().drop(['index'],axis=1)

In [13]:
data = pd.concat([data, names, categorytags, viewed], axis=1)
data.head()

,0,1,2,3,4,5,clubName,categories,viewed
0,401,??중앙동아리 정치경제학연구회 수레바퀴가 신입 연인을 모집합니다,https://www.koreapas.com/bbs/view.php?id=circl...,[],??중앙동아리 정치경제학연구회 수레바퀴가 신입 연인을 모집합니다!\n\r\n?수레바...,중앙동아리 정치경제학연구회 수레바퀴가 신입 연인을 모집합니다중앙동아리 정치경제학연구...,수레바퀴,"[학회, 경영]",74
1,402,[2020 YOUNG TIGERS 신입단원 모집 D-day],https://www.koreapas.com/bbs/view.php?id=circl...,['https://img.koreapas.com/i/de478d7/resize'],[2020 YOUNG TIGERS 신입단원 모집]\n\r\n안녕하십니까 고려대학교 ...,2020 YOUNG TIGERS 신입단원 모집 Dday 2020 YOUNG TIGE...,Young Tigers,"[운동, 동아리]",108
2,403,??[KULSOM D-1] 마감 하루 전!!??????,https://www.koreapas.com/bbs/view.php?id=circl...,['https://img.koreapas.com/i/6f7ae51/resize'],??지원 마감 하루 전입니다! \n\r\n??한 학기 같이 즐거운 추억 만들어요! ...,KULSOM D1 마감 하루 전!! 지원 마감 하루 전입니다! 한 학기 같이 즐...,KULSOM,"[학회, 경영]",98
3,404,[柔道部] 유도부 상시모집 ??,https://www.koreapas.com/bbs/view.php?id=circl...,"['https://img.koreapas.com/i/5b0b384/resize', ...",??고대 유도부 신입부원 상시모집??\r\n????아무나 와라 강하게 키워줄게???...,유도부 상시모집 고대 유도부 신입부원 상시모집 아무나 와라 강하게 키워줄게 고...,유도부,"[동아리, 운동]",86
4,405,[D-3]??KU-HOPE 2020년 1학기 리크루팅!??,https://www.koreapas.com/bbs/view.php?id=circl...,['https://img.koreapas.com/i/e01d297/resize'],??재난에서 희망으로! 안녕하세요 고려대학교 집수리봉사동아리 KU-HOPE 입니다....,D3KUHOPE 2020년 1학기 리크루팅! 재난에서 희망으로! 안녕하세요 고려대학...,KU-HOPE,"[봉사, 동아리]",119


In [14]:
data.to_csv('clubs_200_withviewcnt.csv')

## 동아리 카테고리 추출

In [33]:
alll = []
for _ in data.categories:
    alll+=_

In [38]:
alll = list(set(alll)) #모집?비거니즘??
print(alll)

['봉사', '경영', '대외활동', '음악', '스터디', '사회', '언어', '모집', '예술', '학회', '운동', '동아리', '친목', '연합', 'IT', '비거니즘', '노래', '종교']


# Simple Recommendation System 구현
## 1. 유저가 직접 카테고리 선택 -> 조회수 랭크
고른 카테고리들로 filtering 한 뒤, 조회수로 랭킹

In [67]:
def simple_recommendation_uchoose():
    print('[봉사, 경영, 대외활동, 음악, 스터디, 사회, 언어, 모집, 예술, 학회, 운동, 동아리, 친목, 연합, IT, 비거니즘, 노래, 종교] 중 선택(복수선택 가능)')
    print()
    selected = input().split(' ')
    
    ## 사용자가 고른 태그에 속하는 동아리 추출
    data_selected = data.loc[data.categories.apply(lambda y : any(x in y for x in selected))]
    
    ## 조회수별로 정렬
    selected_ordered = data_selected.sort_values(by='viewed', ascending=False)
    
    print()
    print('당신이 고른 카테고리에 해당되는 동아리를 인기순으로 나열')
    return selected_ordered.clubName.reset_index().clubName

In [68]:
simple_recommendation_uchoose()

[봉사, 경영, 대외활동, 음악, 스터디, 사회, 언어, 모집, 예술, 학회, 운동, 동아리, 친목, 연합, IT, 비거니즘, 노래, 종교] 중 선택(복수선택 가능)

음악 예술

당신이 고른 카테고리에 해당되는 동아리를 인기순으로 나열


0                    합창단
1                  고전기타부
2                  TRUSS
3                    ATP
4                     혼미
5                     혼미
6                  중앙락밴드
7                    프릭스
8                  온 보이싱
9                    관악부
10                    혼미
11                    혼미
12    Talk Through Piano
13                   합창단
14                 성우동아리
15                  SNAP
16                 양지바른곳
17                문화예술체험
18                   로그스
19                영연 연극반
20                 고대문학회
21                   합창단
22               예술비평연구회
23                영연 연극반
24                극예술연구회
25              soulmate
26                 고대풍물패
27                  하날다래
28                   로그스
29        Partyproviders
30                 중앙락밴드
31                   로그스
32                   로그스
33                    혼미
34                    돌빛
35                 중앙락밴드
36                 고대농악대
37                  온보이싱
Name: clubName, dtype: object

## 2. 유저정보에 알맞게 매핑 -> 조회수 랭크

In [71]:
print(alll)

['봉사', '경영', '대외활동', '음악', '스터디', '사회', '언어', '모집', '예술', '학회', '운동', '동아리', '친목', '연합', 'IT', '비거니즘', '노래', '종교']


### <학년>
1학년 => 동아리, 친목, 연합
2학년 => 동아리,대외활동,봉사
3학년 => 대외활동, 봉사
4학년 => 학회, 스터디, 공모전(추가하면 좋을 것 같음)


### <여자/남자>
여자 => 음악, 예술, 언어
남자 => 운동, 음악, IT


### <학과-대학별로?>
정보대학, 정보보호학부, 공과대학, 이과대학 => IT
정경대학, 경영대학 => 경영, 대외활동
간호대학, 보건과학대학 => 사회
국제학부 => 언어
디자인조형학부 => 예술
미디어학부 => 경영, 대외활동, 예술, 음악
문과대학, 사범대학 => 대외활동, 사회, 언어
생명과학대학, 의과대학 => 봉사?????


In [74]:
graded = {'1학년': ['동아리', '친목', '연합'],
'2학년' : ['동아리','대외활동','봉사'],
'3학년' : ['대외활동', '봉사'],
'4학년' : ['학회', '스터디', '공모전']}

    
sexd = {'여자' : ['음악','예술', '언어'],
'남자' : ['운동', '음악', 'IT']}


colleged = {'정보대학' : ['IT'],
            '정보보호학부' : ['IT'],
            '공과대학' : ['IT'],
            '이과대학' : ['IT'],
            '정경대학' :  ['경영', '대외활동'], 
            '경영대학' :  ['경영', '대외활동'], 
            '간호대학':['사회'], 
            '보건과학대학':['사회'],
            '국제학부' : ['언어'],
            '디자인조형학부' : ['예술'],
            '미디어학부' : ['경영', '대외활동', '예술', '음악'],
            '문과대학' : ['대외활동', '사회', '언어'] , 
            '사범대학' : ['대외활동', '사회', '언어'] ,
            '생명과학대학':['봉사'], 
            '의과대학' : ['봉사']}


In [76]:
def simple_recommendation_map():
    print('당신의 학년은?')
    grade = input()
    print()

    print('여자/남자')
    sex = input()
    print()

    print('단과대학')
    college = input()
    
    selected = list(set(graded[grade]+sexd[sex]+colleged[college]))
    
    ## 사용자정보에 매핑하여 속하는 동아리 추출
    data_selected = data.loc[data.categories.apply(lambda y : any(x in y for x in selected))]
    
    ## 조회수별로 정렬
    selected_ordered = data_selected.sort_values(by='viewed', ascending=False)
    
    print()
    print('당신에게 어울리는 동아리를 인기순으로 나열')
    return selected_ordered.clubName.reset_index().clubName

In [77]:
simple_recommendation_map()

당신의 학년은?
4학년

여자/남자
여자

단과대학
정보대학

당신에게 어울리는 동아리를 인기순으로 나열


0      사이버법정책학회
1           코기토
2          수레바퀴
3           합창단
4         고전기타부
         ...   
96          SBC
97        쿠리에이터
98         온보이싱
99      CREAKET
100       쿠리에이터
Name: clubName, Length: 101, dtype: object